In [44]:
import os
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import datasets
import os
import cv2
import keras
from keras import ops

In [2]:
dataset_train = datasets.load_dataset("Dataset/Train")

Resolving data files:   0%|          | 0/61003 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset_test = datasets.load_dataset("Dataset/Test")

Resolving data files:   0%|          | 0/8990 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset_train = dataset_train['train'].shuffle()
dataset_train = dataset_train.train_test_split(test_size=0.1)
dataset_val = dataset_train['test']
dataset_train = dataset_train['train']

dataset_train = (dataset_train).with_format("tf")
dataset_val = (dataset_val).with_format("tf")

dataset_test = (dataset_test['train']).with_format("tf")

In [5]:
print(dataset_val.shape)
print(dataset_train.shape)
print(dataset_test.shape)

(6101, 2)
(54902, 2)
(8990, 2)


In [6]:
X_val = dataset_val['image']
y_val = dataset_val['label']

X_train = dataset_train['image']
y_train = dataset_train['label']

X_test = dataset_test['image']
y_test = dataset_test['label']

In [7]:
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32).flatten()
X_test = np.asarray(X_test, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.int32).flatten()
X_val = np.asarray(X_val, dtype=np.float32)
y_val = np.asarray(y_val, dtype=np.int32).flatten()

In [8]:
print(X_test.shape)

(8990, 64, 64, 3)


In [9]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=True)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [10]:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 64, 64, 3) (64,)
1 (64, 64, 64, 3) (64,)
2 (64, 64, 64, 3) (64,)
3 (64, 64, 64, 3) (64,)
4 (64, 64, 64, 3) (64,)
5 (64, 64, 64, 3) (64,)
6 (64, 64, 64, 3) (64,)


In [11]:
device = "/CPU:0"

def train_part34(model_init, optimizer_init, print_every, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init
        optimizer = optimizer_init
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print(template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [47]:
@keras.saving.register_keras_serializable(package="SleepPoseClassification")
class SleepNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, channel_3, fc1, num_classes):
        input_shape = (64, 64, 3)
        super(SleepNet, self).__init__()

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        
        self.conv11 = tf.keras.layers.Conv2D(channel_1, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv12 = tf.keras.layers.Conv2D(channel_1, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv21 = tf.keras.layers.Conv2D(channel_2, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv22 = tf.keras.layers.Conv2D(channel_2, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv31 = tf.keras.layers.Conv2D(channel_3, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv32 = tf.keras.layers.Conv2D(channel_3, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        
        self.fc1 = tf.keras.layers.Dense(fc1, kernel_initializer=initializer,activation='relu')
        
        
        self.fc2 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer)
        
        self.softmax = tf.keras.layers.Softmax()
    
    def call(self, input_tensor, training=False):
        
        x = self.conv11(input_tensor)
        x = self.conv12(x)
        
        x = self.maxpool(x)
        x = self.conv21(x)
        x = self.conv22(x)
        
        x = self.maxpool(x)
        x = self.conv31(x)
        x = self.conv32(x)
        
        x = self.maxpool(x)
        x = self.flatten(x)
        
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.softmax(x)       

        return x

In [13]:
# v1
# channel_1 = 16
# channel_2 = 32
# channel_3 = 64
# fc1 = 512
# num_classes = 5

In [14]:
channel_1 = 32
channel_2 = 64
channel_3 = 128
fc1 = 512
num_classes = 3

In [77]:
model = SleepNet(channel_1, channel_2, channel_3, fc1, num_classes)
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.summary()

Model: "sleep_net_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_4 (Softmax)             │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [66]:
acc_test = 0
while (acc_test < 0.95):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val), batch_size=64)
    _, acc_test = model.evaluate(X_test, y_test)
    print(acc_test)
    
    
model.evaluate(X_test,y_test)
model.summary()

858/858 ━━━━━━━━━━━━━━━━━━━━ 154s 176ms/step - loss: 265.2583 - sparse_categorical_accuracy: 0.8272 - val_loss: 0.1308 - val_sparse_categorical_accuracy: 0.9420
281/281 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.5212 - sparse_categorical_accuracy: 0.7912
0.6862068772315979
858/858 ━━━━━━━━━━━━━━━━━━━━ 148s 172ms/step - loss: 0.1443 - sparse_categorical_accuracy: 0.9391 - val_loss: 0.1143 - val_sparse_categorical_accuracy: 0.9482
281/281 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - loss: 0.6428 - sparse_categorical_accuracy: 0.7388
0.7335928678512573
858/858 ━━━━━━━━━━━━━━━━━━━━ 157s 183ms/step - loss: 0.1342 - sparse_categorical_accuracy: 0.9444 - val_loss: 0.1117 - val_sparse_categorical_accuracy: 0.9515
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.2718 - sparse_categorical_accuracy: 0.9149
0.7552836537361145
858/858 ━━━━━━━━━━━━━━━━━━━━ 147s 171ms/step - loss: 0.1244 - sparse_categorical_accuracy: 0.9504 - val_loss: 0.1160 - val_sparse_categorical_accuracy: 0.9551
281/281 ━━━━━━━━━━━━

Model: "sleep_net_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │         1,539 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_3 (Softmax)             │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,966,728 (34.21 MB)

 Trainable params: 4,483,363 (17.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,483,365 (17.10 MB)

In [82]:
model.evaluate(X_val,y_val)

191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.1118 - sparse_categorical_accuracy: 0.9575


[0.10561200231313705, 0.9577118754386902]

In [80]:
model.load_weights('SleepNetModelv2_95.weights.h5')

C:\Users\dboga\PycharmProjects\sleep quality using computer vision\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [81]:
model.evaluate(X_test,y_test)

281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.1577 - sparse_categorical_accuracy: 0.9773


[0.2942144572734833, 0.9571746587753296]

In [ ]:
model = SleepNet(channel_1, channel_2, channel_3, fc1, num_classes)


train_part34(model, tf.keras.optimizers.Adam(learning_rate=1e-3), num_epochs=10, is_training=True, print_every=1000)

model.summary()